In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [2]:
from sklearn.preprocessing import StandardScaler

from collections import Counter
from imblearn.combine import SMOTETomek

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import pickle

In [3]:
df = pd.read_csv('/content/drive/MyDrive/My Dataset/ml_olympaid/train.csv')
df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,Amount,Class
0,-0.082423,-0.015885,0.018296,0.105834,0.291156,0.896142,0.416348,1.854701,-1.378885,-1.910856,0.604230,-1.075685,-0.061769,-0.433251,-2.607791,-0.326539,-0.059030,1.140944,-1.716960,-1.399815,0.146446,-1.442996,1.713633,-1.351671,-0.752007,0.174225,8.400,0
1,-0.134807,-0.038678,0.186766,0.061890,1.691469,-0.583866,0.022343,-1.261033,0.060611,1.567038,-0.744162,0.565000,-0.599505,NaN,-0.402509,0.954646,-0.169216,1.949623,0.881305,0.270719,0.200727,0.378079,-0.707367,0.005269,-0.234881,-0.129527,4.200,0
2,0.050357,0.043518,-0.084152,0.121125,1.782521,-0.614403,0.215234,-2.049707,0.018533,0.315355,-0.941150,-0.774718,3.003092,0.273515,1.689802,-1.662092,0.282405,1.831009,1.190881,1.210408,0.754548,1.155251,-0.154011,0.328052,-0.239821,-0.162559,146.958,0
3,0.104938,0.105659,0.441895,-0.272794,-2.273324,2.845803,-1.722852,1.520203,-2.716840,1.817746,1.575395,-0.252826,0.439894,-1.054180,NaN,-0.940693,0.331142,0.482808,1.097569,-0.682473,0.733577,NaN,0.557412,-0.793795,-0.459385,-0.893781,32.256,0
4,NaN,-0.194533,-0.022850,1.017750,0.062345,0.166484,NaN,1.358151,0.234661,1.516270,-1.218913,-0.112761,0.364553,-0.430106,2.085072,0.963968,0.776721,-0.470592,0.236036,1.172562,-1.397078,-1.231025,0.268283,-0.030406,0.258374,0.352570,982.800,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227845 entries, 0 to 227844
Data columns (total 28 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   col_0   216575 non-null  float64
 1   col_1   216487 non-null  float64
 2   col_2   216589 non-null  float64
 3   col_3   216329 non-null  float64
 4   col_4   216404 non-null  float64
 5   col_5   216432 non-null  float64
 6   col_6   216591 non-null  float64
 7   col_7   216303 non-null  float64
 8   col_8   216597 non-null  float64
 9   col_9   216570 non-null  float64
 10  col_10  216486 non-null  float64
 11  col_11  216626 non-null  float64
 12  col_12  216484 non-null  float64
 13  col_13  216509 non-null  float64
 14  col_14  216447 non-null  float64
 15  col_15  216511 non-null  float64
 16  col_16  216601 non-null  float64
 17  col_17  216478 non-null  float64
 18  col_18  216480 non-null  float64
 19  col_19  216395 non-null  float64
 20  col_20  216418 non-null  float64
 21  col_21  21

In [5]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i, t in enumerate(types):
        if 'int' in str(t):
            if (df[cols[i]].min() > np.iinfo(np.int8).min) and (df[cols[i]].max() < np.iinfo(np.int8).max):
                df[cols[i]] = df[cols[i]].astype(np.int8)
                
            elif (df[cols[i]].min() > np.iinfo(np.int16).min) and (df[cols[i]].max() < np.iinfo(np.int16).max):
                df[cols[i]] = df[cols[i]].astype(np.int16)
                
            elif (df[cols[i]].min() > np.iinfo(np.int32).min) and (df[cols[i]].max() < np.iinfo(np.int32).max):
                df[cols[i]] = df[cols[i]].astype(np.int32)
            
            else:
                df[cols[i]] = df[cols[i]].asypes(np.int64)
        elif 'float' in str(t):
            if (df[cols[i]].min() > np.finfo(np.float16).min) and (df[cols[i]].max() < np.finfo(np.float16).max):
                df[cols[i]] = df[cols[i]].astype(np.float16)
                
            elif (df[cols[i]].min() > np.finfo(np.float32).min) and (df[cols[i]].max() < np.finfo(np.float32).max):
                df[cols[i]] = df[cols[i]].astype(np.float32)
            
            else:
                df[cols[i]] = df[cols[i]].asypes(np.float64)
        elif t == object:
            if cols[i] == 'date':
                #df[cols[i]] = pd.to_datetime(df[cols[i]], format = '%Y-%m-%d')
                pass
            else:
                df[cols[i]] = df[cols[i]].astypes('category')
    return df

In [6]:
df = downcast(df)
print('DataFrame Size: {} mb'.format(np.round(df.memory_usage().sum()/(1024*1024),1)))

DataFrame Size: 12.4 mb


> # Content
### 1. Feature Engineering
    * insert the missing values
### 2. Handle the imbalanced Dataset
    * SmoteTomek
### 3. Model Building - XG Boost
    * pipelines
    * Standardization
    * XGBoost

### 1. Feature Engineering

In [7]:
df.isnull().sum()

col_0     11270
col_1     11358
col_2     11256
col_3     11516
col_4     11441
col_5     11413
col_6     11254
col_7     11542
col_8     11248
col_9     11275
col_10    11359
col_11    11219
col_12    11361
col_13    11336
col_14    11398
col_15    11334
col_16    11244
col_17    11367
col_18    11365
col_19    11450
col_20    11427
col_21    11151
col_22    11337
col_23    11313
col_24    11295
col_25    11367
Amount    11268
Class         0
dtype: int64

In [8]:
for i in df.columns:
    df[i] = np.where(df[i].isnull(), df[i].mean(), df[i])

In [9]:
df.isnull().sum()

col_0     0
col_1     0
col_2     0
col_3     0
col_4     0
col_5     0
col_6     0
col_7     0
col_8     0
col_9     0
col_10    0
col_11    0
col_12    0
col_13    0
col_14    0
col_15    0
col_16    0
col_17    0
col_18    0
col_19    0
col_20    0
col_21    0
col_22    0
col_23    0
col_24    0
col_25    0
Amount    0
Class     0
dtype: int64

### 2. Handle the imbalanced Dataset (Smote + Tomek)


In [10]:
Counter(df['Class'])

Counter({0.0: 227459, 1.0: 386})

In [11]:
%%time
df_X = df.iloc[:,:27]
df_Y = df['Class']

#osus = SMOTETomek(0.80)
osus = SMOTETomek(random_state = 42)
df_X_osus, df_Y_osus = osus.fit_resample(df_X, df_Y)
print("The number of classes before fit {}".format(Counter(df_Y)))
print("The number of classes after fit {}".format(Counter(df_Y_osus)))

The number of classes before fit Counter({0.0: 227459, 1.0: 386})
The number of classes after fit Counter({0.0: 227408, 1.0: 227408})
CPU times: user 54min, sys: 18min 25s, total: 1h 12min 25s
Wall time: 45min 40s


In [24]:
df_X_osus.to_csv('/content/drive/MyDrive/My Dataset/ml_olympaid/df_X_osus.csv')

In [25]:
df_Y_osus.to_csv('/content/drive/MyDrive/My Dataset/ml_olympaid/df_Y_osus.csv')

### 3. Model Building - XGBoost

In [12]:
#Standardization

scaler = StandardScaler()
amount = df_X_osus['Amount']
transform_amount = scaler.fit_transform(amount.values.reshape(-1,1))
df_X_osus.drop('Amount', axis = 1, inplace = True)
df_X_osus['Amount'] = transform_amount
df_X_osus

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,Amount
0,-0.082397,-0.015884,0.018295,0.105835,0.291260,0.895996,0.416260,1.854492,-1.378906,-1.911133,0.604004,-1.076172,-0.061768,-0.433350,-2.607422,-0.326660,-0.059021,1.140625,-1.716797,-1.399414,0.146484,-1.443359,1.713867,-1.351562,-0.751953,0.174194,-0.422755
1,-0.134766,-0.038666,0.186768,0.061890,1.691406,-0.583984,0.022339,-1.260742,0.060608,1.567383,-0.744141,0.564941,-0.599609,0.000332,-0.402588,0.954590,-0.169189,1.949219,0.881348,0.270752,0.200684,0.378174,-0.707520,0.005268,-0.234863,-0.129517,-0.426788
2,0.050354,0.043518,-0.084167,0.121155,1.782227,-0.614258,0.215210,-2.048828,0.018539,0.315430,-0.940918,-0.774902,3.003906,0.273438,1.689453,-1.662109,0.282471,1.831055,1.190430,1.209961,0.754395,1.155273,-0.154053,0.328125,-0.239868,-0.162598,-0.289716
3,0.104919,0.105652,0.441895,-0.272705,-2.273438,2.845703,-1.722656,1.520508,-2.716797,1.817383,1.575195,-0.252930,0.439941,-1.053711,-0.000578,-0.940918,0.331055,0.482910,1.097656,-0.682617,0.733398,-0.000228,0.557617,-0.793945,-0.459473,-0.893555,-0.399849
4,0.000230,-0.194580,-0.022858,1.017578,0.062347,0.166504,-0.000102,1.358398,0.234619,1.516602,-1.218750,-0.112732,0.364502,-0.430176,2.085938,0.963867,0.776855,-0.470703,0.236084,1.172852,-1.397461,-1.231445,0.268311,-0.030411,0.258301,0.352539,0.512830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454811,0.629883,-0.389648,0.404785,-0.198608,-0.637207,-2.296875,0.815430,0.492432,-0.322021,1.425781,1.391602,-0.690430,-1.257812,0.827637,0.641113,0.515137,-0.238281,0.418213,-1.617188,0.065857,0.878906,0.375244,-0.868652,-0.055847,-0.933594,3.302734,-0.119252
454812,-0.075378,-0.742676,-0.945312,0.921875,2.416016,0.300781,-0.332764,-2.132812,-0.892578,-0.031525,0.711914,1.520508,0.810059,-1.221680,0.000824,-0.589844,-2.214844,2.208984,2.630859,1.268555,-0.072510,-0.012886,-0.119385,-0.701660,0.823242,1.690430,1.503313
454813,0.666016,-0.449951,0.681152,0.417480,2.261719,-0.746582,1.191406,1.263672,-2.248047,1.924805,0.842773,-0.229858,1.919922,0.545898,0.375977,-1.494141,0.173584,2.367188,-0.996094,-0.538574,0.902344,-1.395508,-0.050171,-0.463135,-0.671387,4.316406,-0.426788
454814,1.567383,-0.323975,2.000000,0.389160,-1.012695,-3.218750,0.913086,-0.077271,0.666992,1.681641,1.186523,-0.008072,1.689453,1.485352,0.172974,-0.078430,1.052734,0.047211,0.478760,1.028320,-1.651367,-1.716797,-1.535156,-0.812500,-4.453125,9.445312,-0.430821


In [13]:
# Split the dataset into train & test set
df_train_X, df_test_X, df_train_Y, df_test_Y = train_test_split(df_X_osus, df_Y_osus, test_size = 0.4)

In [14]:
%%time
# XGBoost intialization & hyperparameter_tuning

#n_estimators = [int(i) for i in np.linspace(start = 400, stop = 500, num = 2)]
#learning_rate = [np.round(i, 2) for i in np.linspace(start = 0.4, stop = 0.5, num = 2)]
#max_depth = [int(i) for i in np.linspace(start = 3, stop = 6, num = 4)]
#
#
#xgboost_classifier = xgb.XGBClassifier()
#parameter_tree = {
#    #'n_estimators' : n_estimators,
#    #'learning_rate': learning_rate,
#    'n_estimators' : [int(500)],
#    'learning_rate': [np.round(0.4,2)],
#    'max_depth': max_depth
#}
#xgboost_hyper = GridSearchCV(xgboost_classifier, param_grid = parameter_tree, cv = 10, scoring = 'f1')

CPU times: user 15 µs, sys: 5 µs, total: 20 µs
Wall time: 23.1 µs


In [15]:
#%%time
#xgboost_hyper.fit(df_train_X, df_train_Y)

In [16]:
#print(xgboost_hyper.best_params_)
#print(xgboost_hyper.best_score_)

In [17]:
#xgboost_classifier = xgboost_hyper.best_estimator_

In [99]:
xgboost_classifier = xgb.XGBClassifier(learning_rate= 0.4, max_depth = 4, min_child_weight = 2, n_estimators =  500, gamma = 0.05)

In [100]:
%%time
xgboost_classifier.fit(df_train_X, df_train_Y)

CPU times: user 4min, sys: 189 ms, total: 4min
Wall time: 4min


XGBClassifier(gamma=0.05, learning_rate=0.4, max_depth=4, min_child_weight=2,
              n_estimators=500)

In [101]:
%%time
from sklearn.metrics import confusion_matrix

preds = xgboost_classifier.predict(df_test_X)
print(confusion_matrix(df_test_Y, preds))

[[90913    51]
 [    0 90963]]
CPU times: user 2.98 s, sys: 1.03 ms, total: 2.98 s
Wall time: 2.98 s


In [102]:
from sklearn.metrics import f1_score

f1_score(df_test_Y, preds, average='macro')

0.9997196677576596

In [86]:
#with open('/content/drive/MyDrive/DS_Model/model/mlolympaid_xgboost_model_v0.9.pkl', 'rb')as f:
#  test = pickle.load(f)

#test

In [103]:
import pickle

with open('/content/drive/MyDrive/DS_Model/model/mlolympaid_xgboost_model_v1.0.pkl', 'wb')as f:
  pickle.dump(xgboost_classifier, f)

In [ ]:
%%time
## Convert the training datatset into Xgboost DMatrix
#df_train_xgb = xgb.DMatrix(df_train_X, label = df_train_Y)
#df_test_xgb = xgb.DMatrix(df_train_Y, label = df_test_Y)
#
#model_name = '/content/drive/MyDrive/DS_Model/model/mlolympaid_xgboost_model_v0.5.pkl'
#with open(model_name, 'rb') as f:
#      xgboost_classifier_v0 = pickle.load(f)
#
#%%time
#parameter_tree = {
#    'n_estimators' : 500,
#    'learning_rate': 0.4,
#    'max_depth': 5
#}
#
#xgboost_classifier = xgb.train(parameter_tree, df_train_xgb, 30, xgb_model = xgboost_classifier_v0.get_booster())

### 4. Predict the testing data

In [104]:
df_test = pd.read_csv('/content/drive/MyDrive/My Dataset/ml_olympaid/test.csv')
df_test.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,Amount,Class
0,-0.261062,-0.156735,0.363162,0.309271,3.786463,0.483927,0.191678,-1.654865,NaN,2.266075,-0.492404,0.912346,-0.867351,1.341530,-0.316175,-1.694447,NaN,3.086475,0.603014,0.866910,0.811921,-1.015089,0.700047,0.067009,-0.741739,0.005553,3.738,NaN
1,-0.152947,0.055197,NaN,0.740416,-0.802117,-1.037894,-0.288519,-1.677102,0.757999,-0.065762,1.486658,-0.392900,-0.153935,-0.512975,-1.539336,-1.225581,-0.396088,-0.028427,-2.271033,-0.466767,0.213215,1.320200,0.367474,-0.151220,0.015337,-0.365593,189.000,NaN
2,-0.177610,-0.034384,-0.211708,-0.097474,-0.296925,-0.153476,0.219271,-0.068303,0.775062,0.051365,0.052075,1.364155,0.420947,-0.425691,-0.303650,1.247138,-0.852429,0.272741,-0.153360,-1.008760,-0.960012,0.039659,-0.370226,-0.079012,0.336124,-0.300902,50.274,NaN
3,-0.195060,0.061956,NaN,0.332315,-2.104488,0.529858,-1.518700,-2.436104,NaN,NaN,1.453674,0.169627,-0.525165,-0.014417,-1.292647,-0.499606,-0.821363,-1.477820,-1.266610,-0.444808,-0.381993,0.901000,-0.234582,-0.469718,-0.342789,-0.062170,18.270,NaN
4,-0.116169,-0.053329,0.092578,0.132276,-1.146944,-0.850719,-0.140537,-0.710435,1.162181,-1.691776,-0.067209,1.056332,0.266278,-0.361343,-1.650970,-0.255608,-0.167654,0.935178,NaN,0.232456,0.154360,-0.338471,1.450368,-0.376392,NaN,-0.125976,NaN,NaN


In [105]:
df_test.shape

(56962, 28)

In [106]:
preds_test = xgboost_classifier.predict(df_test.iloc[:,:27])

In [107]:
df_test.drop('Class', axis = 1)
df_test['Class'] = preds_test.astype('int')

In [108]:
df_test

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,Amount,Class
0,-0.261062,-0.156735,0.363162,0.309271,3.786463,0.483927,0.191678,-1.654865,NaN,2.266075,-0.492404,0.912346,-0.867351,1.341530,-0.316175,-1.694447,NaN,3.086475,0.603014,0.866910,0.811921,-1.015089,0.700047,0.067009,-0.741739,0.005553,3.738,0
1,-0.152947,0.055197,NaN,0.740416,-0.802117,-1.037894,-0.288519,-1.677102,0.757999,-0.065762,1.486658,-0.392900,-0.153935,-0.512975,-1.539336,-1.225581,-0.396088,-0.028427,-2.271033,-0.466767,0.213215,1.320200,0.367474,-0.151220,0.015337,-0.365593,189.000,0
2,-0.177610,-0.034384,-0.211708,-0.097474,-0.296925,-0.153476,0.219271,-0.068303,0.775062,0.051365,0.052075,1.364155,0.420947,-0.425691,-0.303650,1.247138,-0.852429,0.272741,-0.153360,-1.008760,-0.960012,0.039659,-0.370226,-0.079012,0.336124,-0.300902,50.274,0
3,-0.195060,0.061956,NaN,0.332315,-2.104488,0.529858,-1.518700,-2.436104,NaN,NaN,1.453674,0.169627,-0.525165,-0.014417,-1.292647,-0.499606,-0.821363,-1.477820,-1.266610,-0.444808,-0.381993,0.901000,-0.234582,-0.469718,-0.342789,-0.062170,18.270,0
4,-0.116169,-0.053329,0.092578,0.132276,-1.146944,-0.850719,-0.140537,-0.710435,1.162181,-1.691776,-0.067209,1.056332,0.266278,-0.361343,-1.650970,-0.255608,-0.167654,0.935178,NaN,0.232456,0.154360,-0.338471,1.450368,-0.376392,NaN,-0.125976,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56957,0.084047,-0.020802,-0.081409,-0.896240,-1.325285,1.389398,-1.527830,0.130963,-2.539345,NaN,1.648253,NaN,0.404453,0.894848,0.229331,-0.135068,0.406668,-1.802522,2.432430,0.134685,-0.073045,-0.322342,0.019235,1.311488,NaN,-0.318908,828.996,0
56958,-0.239391,-0.231416,0.174891,-0.165479,-0.908681,-1.028243,0.571144,-0.297936,0.197620,0.510022,-0.994997,-0.287568,-0.596003,NaN,-0.158750,0.148162,0.120934,0.277379,0.088232,0.663750,0.872968,0.148152,-0.413716,-1.122326,-0.403711,0.105387,75.558,0
56959,0.085859,0.070810,0.049099,0.044934,-1.695256,-0.727889,NaN,-1.062768,0.959843,0.115982,-0.359693,-0.567509,0.736718,1.128241,-0.417196,0.117274,0.245924,0.649295,-0.459024,0.034741,0.411993,-0.117699,1.104565,1.075458,-0.311424,-0.176072,286.860,0
56960,-0.227868,-0.068702,0.396898,-0.130612,-0.220883,-0.638713,-0.095068,0.176524,1.404656,2.077805,-0.213148,-0.425506,-0.567028,0.292132,-0.104451,0.630312,0.871013,0.322102,0.642909,-1.089444,-0.038657,0.399090,-0.765263,-0.378194,0.456451,-0.217122,3.738,0


In [109]:
df_test_label = df_test['Class']
df_test_label = df_test_label.reset_index()
df_test_label = df_test_label.rename({'index': 'Id'}, axis = 1)

In [110]:
df_test_label['Id'] = df_test_label['Id']+1

In [111]:
df_test_label

,Id,Class
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
56957,56958,0
56958,56959,0
56959,56960,0
56960,56961,0


In [112]:
df_test_label.to_csv('/content/drive/MyDrive/DS_Model/training_result/test_result_v9.csv', index = False)